### Put 0's as neighbouring values in IITM data

In [527]:
import pandas as pd

In [547]:
dff = pd.read_excel("C:/Users/IITM/Desktop/Datathon/test_auto_down/Hadapsar.xlsx")

In [548]:
dff.columns

Index(['Combined', 'Date', 'D', 'M', 'Time', 'T', 'AQI', 'PM2.5', 'Unnamed: 8',
       'Time.1', 'Date.1', 'D1', 'M1', 'T1', 'PM2.5a', 'PM2.5a.1', 'PM2MAX',
       'PM2MIN'],
      dtype='object')

In [549]:
dff['Date'] = pd.to_datetime(dff['Date.1'])
dff.set_index('Date', inplace=True)  
dff.index       = pd.to_datetime(dff.index) 

In [550]:
raik1 = dff.copy()
raik1 = raik1.resample(rule='1H').mean()
raik1 = raik1[['PM2.5a.1']]
value_nan_raik = raik1[raik1.iloc[:,0] == 0.0]

value_nan_raik['date']        = value_nan_raik.index
value_nan_raik['Newkey1']      = value_nan_raik.date.diff()
value_nan_raik['Newkey']      = value_nan_raik.date.diff().ge(datetime.timedelta(hours=2)).cumsum()

value_nan_raik = value_nan_raik.groupby(['Newkey'])['date'].agg(['first','last','count'])

raik_fill = value_nan_raik[~(value_nan_raik['count'] > 24)]
#print(raik_fill)

filled = pd.DataFrame([])

for i in range(0,len(raik_fill)):
    data = raik_fill.iloc[i,:]
    #print(data['first'])
    val1 = pd.DataFrame(raik1[raik1.index == (pd.to_datetime(data['first']) - datetime.timedelta(hours=1))]['PM2.5a.1'])
    #print(i)
    val2 = np.ones(data[2])*val1.values
    datee = pd.date_range(data['first'],data['first']+datetime.timedelta(hours=int(data[2])),freq='1H')
    #print(datee[])
    for ii in range(0,val2.shape[1]):
        #print(val2[ii])
    #    print(datee[ii])
        val  = pd.DataFrame({'Datetime':[datee[ii]],'PM2.5a.1':[val2[0,ii]]})
        #val  = pd.DataFrame({'Datetime':[data['first']],'val'+str(ii):[(val1.values[0,0])]})
        filled =filled.append(val)
    #print(datee)

c:\users\iitm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\iitm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
c:\users\iitm\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [551]:
filled.index = pd.to_datetime(filled.Datetime)
filled = filled.iloc[:,1:]

In [552]:
dff  = dff.resample(rule='1H').mean()
dff.loc[filled.index,['PM2.5a.1']] = filled

In [553]:
dff.to_csv('C:\\Users\\IITM\\Desktop\\Datathon\\iitm_data_imputition\\hadapsar.csv')

# Data Imputation using IITM data

In [554]:
import pandas as pd
import numpy as np

In [693]:
## df1 is IITM
df1 = pd.read_csv('C:/Users/IITM/Desktop/Datathon/iitm_data_imputition/lohgaon.csv')
## df2 is CLIMO
df2 = pd.read_excel('C:/Users/IITM/Desktop/Datathon/test_auto_down/redataimputation/AmbedkarSoceitycircle.xlsx')

In [694]:
itx1 = df2[df2['PM2MAX'] == 0.0]
df_pm2max = pd.DataFrame([])
for ii in range (0,len(itx1)):
    
    diff       = df1.loc[itx1.index[ii],'PM2MAX'] - df1.loc[itx1.index[ii]-1,'PM2MAX']
    increment  = diff/df1.loc[itx1.index[ii]-1,'PM2MAX']
    fill_value = (increment*df2.loc[itx1.index[ii]-1,'PM2MAX']) + df2.loc[itx1.index[ii]-1,'PM2MAX']
    
    df2.loc[itx1.index[ii],'PM2MAX'] =  fill_value
    
    ff = pd.DataFrame({'index':itx1.index[ii],'PM2MAX':[fill_value]})
    df_pm2max = df_pm2max.append([ff])
df_pm2max.index = df_pm2max['index']

In [695]:
itx2 = df2[df2['PM2MIN'] == 0.0]
df_pm2min = pd.DataFrame([])
for ii in range (0,len(itx2)):
    #fillval = (df2.loc[itx2.index[ii]-1,'PM2MIN'].values)*diffdt + df2.loc[itx2.index-1,'PM2MIN'].values
    #print(itx2.index[ii]-1)
    
    diff       = df1.loc[itx2.index[ii],'PM2MIN'] - df1.loc[itx2.index[ii]-1,'PM2MIN']
    increment  = diff/df1.loc[itx2.index[ii]-1,'PM2MIN']
    fill_value = (increment*df2.loc[itx2.index[ii]-1,'PM2MIN']) + df2.loc[itx2.index[ii]-1,'PM2MIN']
    #for j in range(0,5):
    df2.loc[itx2.index[ii],'PM2MIN'] =  fill_value
    #fill_value2 = (increment*fill_value) + fill_value  
    #val = df2.loc[itx2.index[ii]-1,'PM2MIN']
    #print(itx2.index[ii],fill_value)
    ff        = pd.DataFrame({'index':itx2.index[ii],'PM2MIN':[fill_value]})
    df_pm2min = df_pm2min.append([ff])
df_pm2min.index = df_pm2min['index']

In [696]:
df2.loc[itx1.index,'PM2MAX'] = df_pm2max['PM2MAX']

In [697]:
df_pm2min = df_pm2min.drop_duplicates()

In [698]:
b=df2[['D1','M1','T1','PM2MAX','PM2MIN']]

In [699]:
b

,D1,M1,T1,PM2MAX,PM2MIN
0,1.0,4.0,0.0,23.000000,21.000000
1,1.0,4.0,1.0,23.000000,21.000000
2,1.0,4.0,2.0,23.638889,21.000000
3,1.0,4.0,3.0,24.186508,21.000000
4,1.0,4.0,4.0,23.000000,21.000000
5,1.0,4.0,5.0,23.000000,21.000000
6,1.0,4.0,6.0,24.000000,21.000000
7,1.0,4.0,7.0,24.000000,21.000000
8,1.0,4.0,8.0,24.000000,21.000000
9,1.0,4.0,9.0,24.000000,20.750000


In [700]:
b.to_csv("C:\\Users\\IITM\\Desktop\\Datathon\\Final\\ambedkarsoceitycircle.txt","\t",header=True, columns=['D1','M1','T1','PM2MAX','PM2MIN'])